# Gerador de Reviews

- Treinar um modelo preditivo no texto de todos os reviews
- Gerar alguns review aleatórios

### References
- [
Text Generation With LSTM Recurrent Neural Networks in Python with Keras](https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/)
- [Keras example - lstm_text_generation.py](https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py)
- [textgenrnn - Easily train your own text-generating neural network of any size and complexity on any text dataset with a few lines of code.](https://github.com/minimaxir/textgenrnn)

In [2]:
!pip install unidecode keras

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint, LambdaCallback
from keras.utils import np_utils

from collections import defaultdict

from IPython.display import display, Markdown, clear_output

import numpy as np
import random
import itertools

#!pip install unidecode
from unidecode import unidecode

import util.dataset as dataset

Using TensorFlow backend.


In [4]:
def _normalize_text(text, vocabulary=None):
    text = unidecode(text) # Replace unicode chars with closes ascii representation
    text = text.replace('\r\n', '\n').replace('\r', '\n')
    
    if vocabulary is not None:
        text = ''.join([
            char
            for char in text
            if char in vocabulary
        ])
    return text

In [5]:
vocabulary_prob = defaultdict(int)
num_chars = 0
for review in dataset.read('review'):
    for char in _normalize_text(review['text']):
        vocabulary_prob[char] += 1
        num_chars += 1
vocabulary_prob

KeyboardInterrupt: 

In [6]:
vocabulary_prob_sorted = {
    k: v*100/num_chars
    for v, k in sorted([(v,k) for k, v in vocabulary_prob.items()], reverse=True)
}
vocabulary_prob_sorted

{'\n': 0.3149135779359563,
 ' ': 18.491330631767752,
 '!': 0.24138792795286976,
 '"': 0.06445425397312637,
 '#': 0.0016100746164864441,
 '$': 0.03571398573425893,
 '%': 0.0038386535063792662,
 '&': 0.016801063172848383,
 "'": 0.325595048562403,
 '(': 0.06729479211765936,
 ')': 0.07332493607425629,
 '*': 0.0060388706482852,
 '+': 0.004015369013066802,
 ',': 0.621062284907503,
 '-': 0.11676204395263719,
 '.': 1.3184111268809702,
 '/': 0.027268729852932338,
 '0': 0.0737590889857479,
 '1': 0.06082504106726567,
 '2': 0.0504751847249734,
 '3': 0.034610059235692346,
 '4': 0.02590627511310065,
 '5': 0.04552169635541721,
 '6': 0.013841623854062662,
 '7': 0.010999994873068634,
 '8': 0.01154541310358572,
 '9': 0.013754356937179928,
 ':': 0.03157862471047837,
 ';': 0.009950610197553755,
 '<': 1.0908364610341762e-05,
 '=': 0.0013744539409030622,
 '>': 1.0908364610341762e-05,
 '?': 0.03750841171266015,
 '@': 0.0007188612278215222,
 'A': 0.15081795826612418,
 'B': 0.0907554118851214,
 'C': 0.09291090

In [7]:
SYMBOL_PAD = '∅'
SYMBOL_END = '␃'

vocabulary = ''
vocabulary += SYMBOL_PAD
vocabulary += SYMBOL_END
vocabulary += 'abcdefghijklmnopqrstuvwxyz'
vocabulary += 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
vocabulary += '0123456789'
vocabulary += '\n\t '
vocabulary += '.,:;?!\'"`/|\()[]{}<>~@#$^%&*-+_='

char_to_int = {c: i for i, c in enumerate(vocabulary)}
int_to_char = {i: c for i, c in enumerate(vocabulary)}
vocabulary = set(vocabulary)

for char, prob in vocabulary_prob_sorted.items():
    if char not in vocabulary:
        print('Dataset has character missing from vocabulary:', (char, prob))
        
int_to_char

Dataset has character missing from vocabulary: ('\x7f', 1.0908364610341763e-06)


{0: '∅',
 1: '␃',
 2: 'a',
 3: 'b',
 4: 'c',
 5: 'd',
 6: 'e',
 7: 'f',
 8: 'g',
 9: 'h',
 10: 'i',
 11: 'j',
 12: 'k',
 13: 'l',
 14: 'm',
 15: 'n',
 16: 'o',
 17: 'p',
 18: 'q',
 19: 'r',
 20: 's',
 21: 't',
 22: 'u',
 23: 'v',
 24: 'w',
 25: 'x',
 26: 'y',
 27: 'z',
 28: 'A',
 29: 'B',
 30: 'C',
 31: 'D',
 32: 'E',
 33: 'F',
 34: 'G',
 35: 'H',
 36: 'I',
 37: 'J',
 38: 'K',
 39: 'L',
 40: 'M',
 41: 'N',
 42: 'O',
 43: 'P',
 44: 'Q',
 45: 'R',
 46: 'S',
 47: 'T',
 48: 'U',
 49: 'V',
 50: 'W',
 51: 'X',
 52: 'Y',
 53: 'Z',
 54: '0',
 55: '1',
 56: '2',
 57: '3',
 58: '4',
 59: '5',
 60: '6',
 61: '7',
 62: '8',
 63: '9',
 64: '\n',
 65: '\t',
 66: ' ',
 67: '.',
 68: ',',
 69: ':',
 70: ';',
 71: '?',
 72: '!',
 73: "'",
 74: '"',
 75: '`',
 76: '/',
 77: '|',
 78: '\\',
 79: '(',
 80: ')',
 81: '[',
 82: ']',
 83: '{',
 84: '}',
 85: '<',
 86: '>',
 87: '~',
 88: '@',
 89: '#',
 90: '$',
 91: '^',
 92: '%',
 93: '&',
 94: '*',
 95: '-',
 96: '+',
 97: '_',
 98: '='}

In [8]:
def resize_context(context, size):
    context = list(context)
    if len(context) > size:
        context = context[-size:]
    elif len(context) < size:
        context = [SYMBOL_PAD] * (size-len(context)) + context
    
    return context

def gen_trainning_data(context_size=50, batch_size=50):
    def random_text_slice(text):
        text = _normalize_text(text, vocabulary=vocabulary)
        offset = random.randint(0, len(text))
        context = resize_context(text[:offset], size=context_size)
        if offset < len(text):
            next_char = text[offset]
        else:
            next_char = SYMBOL_END

        return context, next_char
                
    data = dataset.read('review', 
            repeat=True, 
            map=lambda review: random_text_slice(review['text']),
            batch_size=batch_size
    )
    #return data
    for batch in data:
        #print(batch)
        context = [[char_to_int[z] for z in x[0]] for x in batch]
        next_char = [char_to_int[x[1]] for x in batch]
        
        context = np_utils.to_categorical(context, len(vocabulary))
        next_char = np_utils.to_categorical(next_char, len(vocabulary))
        yield context, next_char

#next(gen_trainning_data(context_size=3, batch_size=2))

In [9]:
def build_model(units=(512, 512, 512), dropout=(0.2, 0.2, 0.2), context_size=50):

    model = Sequential()
    for i, (lstm_units, lstm_dropout) in enumerate(zip(units, dropout)):
        extra_args = {}
        if i == 0:
            extra_args['input_shape'] = (context_size, len(vocabulary))
        extra_args['return_sequences'] = i < len(units) - 1


        model.add(LSTM(lstm_units, **extra_args))
        if lstm_dropout != 0:
            model.add(Dropout(lstm_dropout))
            
    model.add(Dense(len(vocabulary), activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

model = build_model()
#model.load_weights('review-generator-weights-126-1.4518.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50, 512)           1253376   
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 512)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 50, 512)           2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 50, 512)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 99)                50787     
Total para

In [25]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def gen_texts(prefixes=['']*10, max_len=100, temperature=0.1):
    context_size = model.input_shape[1]
    result = []
    while len(prefixes) != 0:
        x = [resize_context(prefix, context_size) for prefix in prefixes]
        x = np.array([
            [char_to_int[char] for char in resize_context(prefix, context_size)]
            for prefix in prefixes
        ])
        x = np_utils.to_categorical(x, len(vocabulary))
        
        predictions = model.predict(x, verbose=0)
        predictions = [int_to_char[sample(p, temperature=temperature)] for p in predictions]
        
        result += [
            prefix + ('…' if next_char != SYMBOL_END else '')
            for prefix, next_char in zip(prefixes, predictions)
            if next_char == SYMBOL_END or len(prefix) >= max_len
        ]
        prefixes = [
            prefix + next_char
            for prefix, next_char in zip(prefixes, predictions)
            if next_char != SYMBOL_END and len(prefix) < max_len
        ]
    return result
        
def gen_text(prefix='', max_len=100, temperature=0.1):
    return gen_texts(prefixes=[prefix], max_len=max_len, temperature=temperature)[0]
        
print('\n==========\n'.join(gen_texts(max_len=300)))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


I have been to a great staff and the staff is good.  The service was good and the service was good.  I have been to the strip and they have a great staff and the service is always good.  The service was good and the service was good. The service was good and the service was good.  I have been to the…
I have been to the food and they have a great service and the service was good.  The service was good and the food is always good.  The service was good and the prices are great and the service was good.  The service was good and the service was good.  The service was good and the service was good. …
The service is great and the prices are good and the prices are great and the service was always good and the service was good.  The service was good and the service is always good and the service was good. The service was good and the food is always good.  I have been to the strip and they have a g…
I have been to the food and the staff is always good.  The service was good and the prices are

In [11]:
checkpoint_callbacks = ModelCheckpoint('review-generator-weights-{epoch:02d}-{loss:.4f}.h5', monitor='loss', verbose=1, save_best_only=True, mode='min')
eval_callback = LambdaCallback(on_epoch_end=lambda epoch, logs: print('\n ---x---\n'.join([''] + gen_texts(['']*5, max_len=50) + [''])))

# fit the model
model.fit_generator(
    gen_trainning_data(), 
    epochs=1000, 
    steps_per_epoch=100,
    callbacks=[
        checkpoint_callbacks,
        eval_callback
    ])

Epoch 1/1000
100/100 [==============================] - 16s 161ms/step - loss: 3.2710

Epoch 00001: loss improved from inf to 3.27098, saving model to review-generator-weights-01-3.2710.h5

 ---x---
                                                  …
 ---x---
                                                  …
 ---x---
                     e                            …
 ---x---
                                                  …
 ---x---
                                                  …
 ---x---

Epoch 2/1000
100/100 [==============================] - 15s 146ms/step - loss: 3.1499

Epoch 00002: loss improved from 3.27098 to 3.14992, saving model to review-generator-weights-02-3.1499.h5

 ---x---
                                                  …
 ---x---
                                                  …
 ---x---
                                                  …
 ---x---
                                    e             …
 ---x---
         e                                      

100/100 [==============================] - 15s 147ms/step - loss: 2.1892

Epoch 00017: loss improved from 2.19686 to 2.18918, saving model to review-generator-weights-17-2.1892.h5

 ---x---
The stall the sthe sto the store the sto the stell…
 ---x---
The sto the sting the sto the sto the stall the st…
 ---x---
The sto the stell the stall the sto the sto the st…
 ---x---
The stall and the stall and the sto the stome the …
 ---x---
I the sto the stall and the sto a was the sto the …
 ---x---

Epoch 18/1000
100/100 [==============================] - 15s 147ms/step - loss: 2.1270

Epoch 00018: loss improved from 2.18918 to 2.12703, saving model to review-generator-weights-18-2.1270.h5

 ---x---
The some the some and the sour and the paste the p…
 ---x---
I was the sour and the sour and the prace and the …
 ---x---
The service and the past and the porle the really …
 ---x---
The sare the save and and the past and the some an…
 ---x---
The sour and the sour and the come the prace and t…
 ---

100/100 [==============================] - 15s 147ms/step - loss: 1.8312

Epoch 00034: loss improved from 1.83389 to 1.83125, saving model to review-generator-weights-34-1.8312.h5

 ---x---
I come the food and the service and the service an…
 ---x---
I was a love the food and the staff and the was a …
 ---x---
I have a we had the staff and the staff and the st…
 ---x---
I was a come the food and the from the food and th…
 ---x---
I was a come the food and the food and the staff a…
 ---x---

Epoch 35/1000
100/100 [==============================] - 15s 147ms/step - loss: 1.8130

Epoch 00035: loss improved from 1.83125 to 1.81299, saving model to review-generator-weights-35-1.8130.h5

 ---x---
I have a staff and the staff and the staff and the…
 ---x---
I have a staff and the staff and the service was s…
 ---x---
The staff and the service and the staff and the st…
 ---x---
I was service and the staff and the staff and the …
 ---x---
I have any the staff and the staff and the staff a…
 ---

100/100 [==============================] - 15s 147ms/step - loss: 1.6769

Epoch 00051: loss did not improve from 1.67539

 ---x---
I will be the place is also the staff was great se…
 ---x---
I will be the staff was great and the chicken serv…
 ---x---
The service was great service was a price is great…
 ---x---
The service is also the place was a great service …
 ---x---
The service was great service is always great serv…
 ---x---

Epoch 52/1000
100/100 [==============================] - 15s 147ms/step - loss: 1.6725

Epoch 00052: loss improved from 1.67539 to 1.67249, saving model to review-generator-weights-52-1.6725.h5

 ---x---
I was the service is a great and the service was a…
 ---x---
I love the service is a great and the service was …
 ---x---
I was a great and the service is a great and the s…
 ---x---
I was the service was a great and the service and …
 ---x---
I was a great and the service was a great and the …
 ---x---

Epoch 53/1000
100/100 [==============================

100/100 [==============================] - 15s 147ms/step - loss: 1.6075

Epoch 00069: loss improved from 1.62307 to 1.60753, saving model to review-generator-weights-69-1.6075.h5

 ---x---
The place is a couple and the place is a great and…
 ---x---
The place is a better and the place is a great and…
 ---x---
The chicken and the price is a couple to the price…
 ---x---
The food is a great and the place is a great and t…
 ---x---
The food is a couple and the could and the still a…
 ---x---

Epoch 70/1000
100/100 [==============================] - 15s 147ms/step - loss: 1.6004

Epoch 00070: loss improved from 1.60753 to 1.60039, saving model to review-generator-weights-70-1.6004.h5

 ---x---
I had the service was a but the service was a grea…
 ---x---
I had the staff was a but the service was a great …
 ---x---
I had the food was a but the service was a great f…
 ---x---
I had the service was a but the service was a grea…
 ---x---
I had the service was a great service was a great …
 ---

...

100/100 [==============================] - 15s 147ms/step - loss: 1.3325

Epoch 00915: loss did not improve from 1.23443

 ---x---
This place was great and the service was great.  I…
 ---x---
I was a great place to go to this place and the sa…
 ---x---
This place was great.  I would have been to the st…
 ---x---
I was able to try the stars and the service was go…
 ---x---
This is the best place to go to the strip and the …
 ---x---

Epoch 916/1000
100/100 [==============================] - 15s 147ms/step - loss: 1.2740

Epoch 00916: loss did not improve from 1.23443

 ---x---
This place is a great place to go to the store.  T…
 ---x---
This is the best place to go to the food and the s…
 ---x---
This place is a great place to go to the store and…
 ---x---
This place is a great place to go to the food and …
 ---x---
I have been to this place and the service was grea…
 ---x---

Epoch 917/1000
100/100 [==============================] - 15s 148ms/step - loss: 1.2997

Epoch 00917: loss did 

100/100 [==============================] - 15s 147ms/step - loss: 1.3034

Epoch 00933: loss did not improve from 1.23443

 ---x---
This place is a great service. The service was ama…
 ---x---
The service was great and the service was great. T…
 ---x---
I was a little honestly and I was a lot of the sta…
 ---x---
The service was great and the service was great.  …
 ---x---
The service was amazing and the service was great.…
 ---x---

Epoch 934/1000
100/100 [==============================] - 15s 148ms/step - loss: 1.3167

Epoch 00934: loss did not improve from 1.23443

 ---x---
The service was great.  The service was amazing an…
 ---x---
The service was great and the service was great.  …
 ---x---
The staff is a great place to go to the strawberry…
 ---x---
I was a little happy here and the service was amaz…
 ---x---
The service was amazing.  The service was great an…
 ---x---

Epoch 935/1000
100/100 [==============================] - 15s 147ms/step - loss: 1.2919

Epoch 00935: loss did 

100/100 [==============================] - 15s 147ms/step - loss: 1.2871

Epoch 00951: loss did not improve from 1.23443

 ---x---
I have been to the started the staff and the servi…
 ---x---
I have been to the start and the service was great…
 ---x---
I have been to this place to go with the street an…
 ---x---
I have been to this place to get the same thing th…
 ---x---
I have been to this place to go to the staff and t…
 ---x---

Epoch 952/1000
100/100 [==============================] - 15s 148ms/step - loss: 1.2986

Epoch 00952: loss did not improve from 1.23443

 ---x---
I have been to this place the food was awesome and…
 ---x---
I have been to the food and the service was amazin…
 ---x---
I have been to this place to go to the start of th…
 ---x---
I have been to this place to go to the store and t…
 ---x---
I have been to this place the staff and the servic…
 ---x---

Epoch 953/1000
100/100 [==============================] - 15s 147ms/step - loss: 1.2803

Epoch 00953: loss did 

100/100 [==============================] - 15s 147ms/step - loss: 1.3343

Epoch 00969: loss did not improve from 1.23443

 ---x---
I was a little staff and the service was good. The…
 ---x---
I was a little salad and the staff is always good.…
 ---x---
I have been to the food and the food was great. Th…
 ---x---
I was a little staff and the staff is always good.…
 ---x---
I was a little best for a great staff and the serv…
 ---x---

Epoch 970/1000
100/100 [==============================] - 15s 147ms/step - loss: 1.3101

Epoch 00970: loss did not improve from 1.23443

 ---x---
The food was amazing and the food is always good. …
 ---x---
I have been to this place to the food and the serv…
 ---x---
I was a little beautiful and the service was also …
 ---x---
I have been to this place and the service was amaz…
 ---x---
I would have been here a few times and the service…
 ---x---

Epoch 971/1000
100/100 [==============================] - 15s 147ms/step - loss: 1.3247

Epoch 00971: loss did 

100/100 [==============================] - 15s 147ms/step - loss: 1.3049

Epoch 00987: loss did not improve from 1.23443

 ---x---
I have been to the best service was a bit over the…
 ---x---
The service was amazing!  The food is awesome and …
 ---x---
The service was good and the food was awesome and …
 ---x---
The food was awesome and the food was awesome and …
 ---x---
The food was awesome and the food was amazing!  Th…
 ---x---

Epoch 988/1000
100/100 [==============================] - 15s 148ms/step - loss: 1.3349

Epoch 00988: loss did not improve from 1.23443

 ---x---
The service was good and the service was great and…
 ---x---
I had the best place to try the best place to be a…
 ---x---
The best place to go with a strip to the best plac…
 ---x---
I was able to go to the best place to go to the st…
 ---x---
I have to say the service was absolutely awesome. …
 ---x---

Epoch 989/1000
100/100 [==============================] - 15s 147ms/step - loss: 1.3006

Epoch 00989: loss did 

In [17]:
for x in gen_texts(temperature=0.3, max_len=500):
    print(x)
    print('-------x--------')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


The best place for my party of the most satisfied and they have a great service.  They are always good and the service was always good and the prices are always very delicious and the prices are great! The prices are good as well. The food is always good and the place is awesome and the prices are very friendly and the service was very professional. The service was good and the place is always friendly and friendly. The service was good and the food is okay. I love the staff so I would go to thi…
-------x--------
The waiter was always good!  The service is always good and the best food is always good and the service was awesome. They are very so good! The service was friendly and the prices are friendly and the prices are good and the staff is good and the complaint was always a bit on the same time for the complaint in the patio and a disappointing at the best place to go with the park and the cheap bowl to the price and they were good and the prices are very good! I have been to the 

In [24]:
prefix_len = 80
for autocomplete_text in dataset.read('review', map=lambda review:_normalize_text(review['text'])):
    prefix = ""
    for char in [""] + list(autocomplete_text):
        prefix += char
        pretty_prefix = '[empty]' if prefix == '' else prefix
        if len(pretty_prefix) > prefix_len: 
            pretty_prefix = '…' + pretty_prefix[len(pretty_prefix)-prefix_len+1:]
        completions = gen_texts(prefixes=[prefix]*5, max_len=len(prefix)+30, temperature=0.3)
        clear_output()
        display(
            Markdown('### ' + pretty_prefix),
            Markdown('\n'.join(' - ' + ('…' if prefix else '') + x[len(prefix):] for x in (completions))))


### …rted to suck and their portion sizes got cut in half. I have a huge problem wit

 - …h a bartender and was a great …
 - …h the park to the table and th…
 - …h a lot of produce for a coupl…
 - …h the parking and the food is …
 - …h a great staff and they did a…

KeyboardInterrupt: 

-----------
\





## Considerações de Performance

Essa implementação foi feita com simplicidade e legibilidade em mente, e é terrivelmente ineficiente.

### Treino
Para começar, a cada review, um único trecho de 50 caracteres é extraido e usado para treinar um único caracter de saida.

Em outras palavras, este código processa uma função _extremamente_ complexa, com 5 milhões de parametros, para simplesmente descartar os resultados intermediários e considerar apenas um entre centenas de caracteres no cálculo de loss.

Uma solução melhor deveria levar em consideração a estrutura do LSTM: É possível prever o próximo caracter para cada um dos elementos da sequencia e usar predições intermediárias no cálculo de loss, efetivamente treinando todas as letras da sequencia simultaneamente.

### Predição
A predição também é feita de forma idiota. Para cada caractere a ser predito, o trecho completo dos 50 caracteres anteriores é alimentado na rede neural, causando uma quantidade enorme de calculos redundantes.

Uma versão mais eficiente deveria preservar o estado da LSTM e calcular apenas a predição do próximo caractere.





## Usos

Gerar texto que se parece com uma review é divertido, mas não é terrívelmente útil.

Um uso mais interessante é no auxilio a digitação, especialmente em celulares. Isto me lembrou do [Dasher](http://www.inference.org.uk/dasher/) ([Demo](https://www.youtube.com/watch?v=nr3s4613DX8)), um projeto que existe há muitos anos e que certamente não usa redes neurais, mas que talvez poderia ser um bom teclado android.